Objectives: categorize continuous variables:

CALCIUM: 'Calcium_Closest_Osteo','Calcium_Avg_Prior',  'Calcium_Avg_Ever' 
                         
SODIUM:  __'Sodium_Closest_Osteo',__'Sodium_Avg_Prior', 'Sodium_Avg_Ever', 'Sodium_Worst_Prior','Sodium_Worst_Ever'

---> 8 variables


1. __Discretize__ (bins start from 0 to 9) (0 here != missing)
        'Calcium_Closest_Osteo': 'Calcium_Closest_Osteo_decile', 
        'Calcium_Avg_Prior': 'Calcium_Avg_Prior_decile',
        'Calcium_Avg_Ever': 'Calcium_Avg_Ever_decile',

         __'Sodium_Closest_Osteo'__: 'Sodium_Closest_Osteo_decile', --> need special function to recode (i.e. missing = 0, the rest 10 bins start w 1)

        'Sodium_Avg_Prior': 'Sodium_Avg_Prior_decile',
        'Sodium_Avg_Ever': 'Sodium_Avg_Ever_decile',
        'Sodium_Worst_Prior': 'Sodium_Worst_Prior_decile',
        'Sodium_Worst_Ever': 'Sodium_Worst_Ever_decile'

2. __Categorize__: missing = 0, not missing = 1
         'Calcium_Closest_Osteo': 'Calcium_Closest_Osteo_cat', 
         'Calcium_Avg_Prior': 'Calcium_Avg_Prior_cat',
         'Calcium_Avg_Ever': 'Calcium_Avg_Ever_cat',

        'Sodium_Avg_Prior': 'Sodium_Avg_Prior_cat',
        'Sodium_Avg_Ever': 'Sodium_Avg_Ever_cat',
        'Sodium_Worst_Prior': 'Sodium_Worst_Prior_cat',
        'Sodium_Worst_Ever': 'Sodium_Worst_Ever_cat'
        
3. 
    - LR AUC = 81% 
    - RF AUC = 83%
    - baseline = 67% without adding calcium or sodium

In [244]:
import pandas as pd
import numpy as np
from matplotlib import pyplot

from sklearn.model_selection import KFold, cross_val_predict, cross_validate
import sklearn.linear_model as linear_model

from sklearn.cross_validation import cross_val_score
import seaborn as sns 
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix, classification_report, recall_score, precision_score, f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

import matplotlib.pyplot as plt

%matplotlib inline

In [245]:
data_osteo = pd.read_csv('osteo_cleaning_BMI_impute.csv') 
data_osteo.shape

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(61022, 192)

In [3]:
### check # of missing

In [246]:
def check_missing(data, var):
    print(var)
    print('# of missing: ', data[var].isna().sum(), 'out of', data.shape[0])
    print('% of missing: ',  data[var].isna().sum()/data.shape[0], '\n')

In [247]:
var_Calcium = ['Calcium_Closest_Osteo','Calcium_Avg_Prior', 'Calcium_Avg_Ever']
for i in var_Calcium:
    check_missing(data_osteo,i)

Calcium_Closest_Osteo
# of missing:  25454 out of 61022
% of missing:  0.4171282488282914 

Calcium_Avg_Prior
# of missing:  25385 out of 61022
% of missing:  0.4159975090950805 

Calcium_Avg_Ever
# of missing:  6791 out of 61022
% of missing:  0.11128773229327128 



In [248]:
sodium = ['Sodium_Closest_Osteo','Sodium_Avg_Prior', 'Sodium_Avg_Ever', 'Sodium_Worst_Prior','Sodium_Worst_Ever']
for i in sodium:
    check_missing(data_osteo,i)

Sodium_Closest_Osteo
# of missing:  18444 out of 61022
% of missing:  0.30225164694700274 

Sodium_Avg_Prior
# of missing:  19891 out of 61022
% of missing:  0.3259644062797024 

Sodium_Avg_Ever
# of missing:  249 out of 61022
% of missing:  0.004080495558978729 

Sodium_Worst_Prior
# of missing:  19891 out of 61022
% of missing:  0.3259644062797024 

Sodium_Worst_Ever
# of missing:  249 out of 61022
% of missing:  0.004080495558978729 



--> more missing in before diagnosis 

## Seperate into decile 10

CALCIUM: 'Calcium_Closest_Osteo','Calcium_Avg_Prior',  'Calcium_Avg_Ever', 
                         
SODIUM:  'Sodium_Closest_Osteo','Sodium_Avg_Prior', 'Sodium_Avg_Ever',
         'Sodium_Worst_Prior','Sodium_Worst_Ever'

In [249]:
def decile(data, var, new_var):
    '''create 10 bins and discretize data
    '''
    print(pd.qcut(data[var],10).value_counts()) 
    data[new_var] = pd.qcut(data[var],10, labels=False) 
    return pd.qcut(data[var],10, labels=False) 

In [250]:
def pct_rank_qcut(series, n):
    '''special case for errors in qcut
    here missing = 0
    '''
    edges = pd.Series([float(i) / n for i in range(n + 1)])
    f = lambda x: (edges >= x).values.argmax()
    print(series.rank(pct=1).apply(f).value_counts())
    return series.rank(pct=1).apply(f)
data_osteo['Sodium_Closest_Osteo_decile'] = pct_rank_qcut(data_osteo['Sodium_Closest_Osteo'], 10)

0     18444
6      5305
7      4993
5      4965
1      4776
4      4265
2      4185
8      4181
10     3854
3      3182
9      2872
Name: Sodium_Closest_Osteo, dtype: int64


In [251]:
#create discretize var + add to df
dic = {'Calcium_Closest_Osteo': 'Calcium_Closest_Osteo_decile', 
      'Calcium_Avg_Prior': 'Calcium_Avg_Prior_decile',
      'Calcium_Avg_Ever': 'Calcium_Avg_Ever_decile',
#     'Sodium_Closest_Osteo': 'Sodium_Closest_Osteo_decile',
    'Sodium_Avg_Prior': 'Sodium_Avg_Prior_decile',
    'Sodium_Avg_Ever': 'Sodium_Avg_Ever_decile',
    'Sodium_Worst_Prior': 'Sodium_Worst_Prior_decile',
    'Sodium_Worst_Ever': 'Sodium_Worst_Ever_decile'}

for i,j in dic.items():
    decile(data_osteo, i,j) 

(7.8, 8.4]      4434
(9.1, 9.3]      4097
(9.6, 9.9]      3905
(8.4, 8.7]      3834
(8.9, 9.1]      3813
(0.599, 7.8]    3632
(9.4, 9.6]      3521
(8.7, 8.9]      3216
(9.9, 84.0]     3113
(9.3, 9.4]      2003
Name: Calcium_Closest_Osteo, dtype: int64
(9.2, 9.4]        4207
(9.577, 9.8]      3725
(9.054, 9.2]      3725
(8.349, 8.64]     3575
(7.781, 8.349]    3564
(0.749, 7.781]    3564
(8.64, 8.867]     3560
(8.867, 9.054]    3556
(9.8, 37.4]       3402
(9.4, 9.577]      2759
Name: Calcium_Avg_Prior, dtype: int64
(9.225, 9.4]      6363
(9.567, 9.8]      5857
(8.87, 9.067]     5473
(9.067, 9.225]    5427
(7.609, 8.294]    5424
(0.749, 7.609]    5424
(8.294, 8.623]    5423
(8.623, 8.87]     5422
(9.8, 117.883]    4876
(9.4, 9.567]      4542
Name: Calcium_Avg_Ever, dtype: int64
(138.154, 139.0]      5015
(140.393, 141.0]      4983
(141.0, 142.0]        4962
(9.999, 135.333]      4142
(135.333, 137.0]      4129
(139.833, 140.393]    4086
(137.0, 138.154]      4073
(143.0, 1212.0]       33

## Creating category (0 = missing, 1 = non-missing)

In [259]:
def present_absence(data, var, new_var):
    ''' create var: 0 = missing, 1 = non-missing
    '''
    print('# of missing: ', data[var].isna().sum())
        
    not_missing = pd.notna(data[var])
    b = []
    for i in not_missing:
        if i == True:
            b.append(1)
        if i == False:
            b.append(0)

    import collections
    counter=collections.Counter(b)
    print(counter)
    data[new_var] = b

In [260]:
#create absence/present var + add to df
dic = {'Calcium_Closest_Osteo': 'Calcium_Closest_Osteo_cat', 
      'Calcium_Avg_Prior': 'Calcium_Avg_Prior_cat',
      'Calcium_Avg_Ever': 'Calcium_Avg_Ever_cat',
#     'Sodium_Closest_Osteo': 'Sodium_Closest_Osteo_decile',
    'Sodium_Avg_Prior': 'Sodium_Avg_Prior_cat',
    'Sodium_Avg_Ever': 'Sodium_Avg_Ever_cat',
    'Sodium_Worst_Prior': 'Sodium_Worst_Prior_cat',
    'Sodium_Worst_Ever': 'Sodium_Worst_Ever_cat'}

for i,j in dic.items():
    print('var: ', i)
    present_absence(data_osteo, i,j)
    print(' ')

var:  Calcium_Closest_Osteo
# of missing:  25454
Counter({1: 35568, 0: 25454})
 
var:  Calcium_Avg_Prior
# of missing:  25385
Counter({1: 35637, 0: 25385})
 
var:  Calcium_Avg_Ever
# of missing:  6791
Counter({1: 54231, 0: 6791})
 
var:  Sodium_Avg_Prior
# of missing:  19891
Counter({1: 41131, 0: 19891})
 
var:  Sodium_Avg_Ever
# of missing:  249
Counter({1: 60773, 0: 249})
 
var:  Sodium_Worst_Prior
# of missing:  19891
Counter({1: 41131, 0: 19891})
 
var:  Sodium_Worst_Ever
# of missing:  249
Counter({1: 60773, 0: 249})
 


In [261]:
data_osteo.shape

(61022, 207)

In [262]:
#export to dataset
data_osteo.to_csv('osteo_discretized.csv')

In [263]:
data_osteo.columns

Index(['_Imputation_', 'patientid', 'sex', 'Race', 'Ethnicity', 'DOB',
       'ENC_DATE1', 'ENC_DATE2', 'Enc_Mth_Diff', 'Enc_Day_Diff',
       ...
       'Sodium_Avg_Ever_decile', 'Sodium_Worst_Prior_decile',
       'Sodium_Worst_Ever_decile', 'Calcium_Closest_Osteo_cat',
       'Calcium_Avg_Prior_cat', 'Calcium_Avg_Ever_cat', 'Sodium_Avg_Prior_cat',
       'Sodium_Avg_Ever_cat', 'Sodium_Worst_Prior_cat',
       'Sodium_Worst_Ever_cat'],
      dtype='object', length=207)

## Observations 1     (% of missing across subgroups)

In [264]:
def check_across_subgroup(data, col, row):
    return pd.crosstab(index=data[col], columns=data[row], margins = True)/data.shape[0]

In [265]:
var=['Calcium_Closest_Osteo_cat', 'Calcium_Avg_Prior_cat','Calcium_Avg_Ever_cat','Sodium_Avg_Prior_cat',
   'Sodium_Avg_Ever_cat','Sodium_Worst_Prior_cat','Sodium_Worst_Ever_cat']

for i in var:
    print(check_across_subgroup(data_osteo, i, 'sex'), '\n')

sex                               F         M       All
Calcium_Closest_Osteo_cat                              
0                          0.374701  0.042427  0.417128
1                          0.507800  0.075071  0.582872
All                        0.882501  0.117499  1.000000 

sex                           F         M       All
Calcium_Avg_Prior_cat                              
0                      0.371620  0.044377  0.415998
1                      0.510881  0.073121  0.584002
All                    0.882501  0.117499  1.000000 

sex                          F         M       All
Calcium_Avg_Ever_cat                              
0                     0.097834  0.013454  0.111288
1                     0.784668  0.104044  0.888712
All                   0.882501  0.117499  1.000000 

sex                          F         M       All
Sodium_Avg_Prior_cat                              
0                     0.297925  0.028039  0.325964
1                     0.584576  0.089460  0.67

### --> missing(female) > missing(male)

In [266]:
var=['Calcium_Closest_Osteo_cat', 'Calcium_Avg_Prior_cat','Calcium_Avg_Ever_cat','Sodium_Avg_Prior_cat',
   'Sodium_Avg_Ever_cat','Sodium_Worst_Prior_cat','Sodium_Worst_Ever_cat']

for i in var:
    print(check_across_subgroup(data_osteo, i, 'osteo_predict'), '\n')

osteo_predict                     0         1       All
Calcium_Closest_Osteo_cat                              
0                          0.170856  0.246272  0.417128
1                          0.329144  0.253728  0.582872
All                        0.500000  0.500000  1.000000 

osteo_predict                 0         1       All
Calcium_Avg_Prior_cat                              
0                      0.174511  0.241487  0.415998
1                      0.325489  0.258513  0.584002
All                    0.500000  0.500000  1.000000 

osteo_predict                0         1       All
Calcium_Avg_Ever_cat                              
0                     0.009275  0.102012  0.111288
1                     0.490725  0.397988  0.888712
All                   0.500000  0.500000  1.000000 

osteo_predict                0         1       All
Sodium_Avg_Prior_cat                              
0                     0.168382  0.157583  0.325964
1                     0.331618  0.342417  0.67

### ---> in calcium: missing (osteo_positive) > missing (osteo_neg)
### opposite for sodium

## Observation 2:  Among all missing, % of female/male comparing to class labels( osteo_pos/osteo_neg)

In [273]:
def extract_outliers(data, variable, threshold, side):
    if side == "greater":
        print('# of outliers: ', data.loc[data[variable] >= threshold][variable].count())
        return data.loc[data[variable] >= threshold][variable]
    if side == "less":
        print('# of outliers: ', data.loc[data[variable] <= threshold][variable].count())
        return data.loc[data[variable] <= threshold]
    if side == 'equal':
        print('# of outliers: ', data.loc[data[variable] == threshold][variable].count())
        return data.loc[data[variable] == threshold]

In [274]:
def extract_missing_check_subgroup(data, var1, var2, var3):
    print('# of rows: \n', data[var1].value_counts())
    calculate_missing = extract_outliers(data, var1, 0, 'equal')
    print(check_across_subgroup(calculate_missing, var2, var3)) 

In [276]:
var = ['Calcium_Closest_Osteo_cat', 'Calcium_Avg_Prior_cat','Calcium_Avg_Ever_cat','Sodium_Avg_Prior_cat',
   'Sodium_Avg_Ever_cat','Sodium_Worst_Prior_cat','Sodium_Worst_Ever_cat']
for i in var:
    extract_missing_check_subgroup(data_osteo, i, 'sex', 'osteo_predict')
    print(' ') 

# of rows: 
 1    35568
0    25454
Name: Calcium_Closest_Osteo_cat, dtype: int64
# of outliers:  25454
osteo_predict         0         1       All
sex                                        
F              0.366151  0.532136  0.898287
M              0.043451  0.058262  0.101713
All            0.409602  0.590398  1.000000
 
# of rows: 
 1    35637
0    25385
Name: Calcium_Avg_Prior_cat, dtype: int64
# of outliers:  25385
osteo_predict         0         1       All
sex                                        
F              0.375261  0.518062  0.893323
M              0.044239  0.062438  0.106677
All            0.419500  0.580500  1.000000
 
# of rows: 
 1    54231
0     6791
Name: Calcium_Avg_Ever_cat, dtype: int64
# of outliers:  6791
osteo_predict         0         1       All
sex                                        
F              0.070976  0.808128  0.879105
M              0.012369  0.108526  0.120895
All            0.083346  0.916654  1.000000
 
# of rows: 
 1    41131
0    19891


### --> among all the missing, in calcium: female(positive) is dominant
### sodium is opposite

In [278]:
var = ['Calcium_Closest_Osteo_cat', 'Calcium_Avg_Prior_cat','Calcium_Avg_Ever_cat','Sodium_Avg_Prior_cat',
   'Sodium_Avg_Ever_cat','Sodium_Worst_Prior_cat','Sodium_Worst_Ever_cat']
for i in var:
    extract_missing_check_subgroup(data_osteo, i,'Race_Cat_combine' , 'osteo_predict')
    print(' ') 

# of rows: 
 1    35568
0    25454
Name: Calcium_Closest_Osteo_cat, dtype: int64
# of outliers:  25454
osteo_predict            0         1       All
Race_Cat_combine                              
black             0.111810  0.162057  0.273867
others            0.061876  0.059480  0.121356
white             0.235916  0.368861  0.604777
All               0.409602  0.590398  1.000000
 
# of rows: 
 1    35637
0    25385
Name: Calcium_Avg_Prior_cat, dtype: int64
# of outliers:  25385
osteo_predict            0         1       All
Race_Cat_combine                              
black             0.114201  0.167185  0.281387
others            0.066496  0.080126  0.146622
white             0.238802  0.333189  0.571991
All               0.419500  0.580500  1.000000
 
# of rows: 
 1    54231
0     6791
Name: Calcium_Avg_Ever_cat, dtype: int64
# of outliers:  6791
osteo_predict            0         1       All
Race_Cat_combine                              
black             0.014873  0.253865  0

### --> among all the missing, white(positive) 

In [279]:
var = ['Calcium_Closest_Osteo_cat', 'Calcium_Avg_Prior_cat','Calcium_Avg_Ever_cat','Sodium_Avg_Prior_cat',
   'Sodium_Avg_Ever_cat','Sodium_Worst_Prior_cat','Sodium_Worst_Ever_cat']
for i in var:
    extract_missing_check_subgroup(data_osteo, i,'Age_combine' , 'osteo_predict')
    print(' ') 

# of rows: 
 1    35568
0    25454
Name: Calcium_Closest_Osteo_cat, dtype: int64
# of outliers:  25454
osteo_predict         0         1       All
Age_combine                                
from_30_to_50  0.050915  0.058812  0.109727
from_50_to_70  0.186847  0.244323  0.431170
from_70_to_80  0.096841  0.152039  0.248880
less_than_30   0.008329  0.007700  0.016029
more_than_80   0.066669  0.127524  0.194193
All            0.409602  0.590398  1.000000
 
# of rows: 
 1    35637
0    25385
Name: Calcium_Avg_Prior_cat, dtype: int64
# of outliers:  25385
osteo_predict         0         1       All
Age_combine                                
from_30_to_50  0.052039  0.061257  0.113295
from_50_to_70  0.190270  0.256136  0.446405
from_70_to_80  0.099389  0.140358  0.239748
less_than_30   0.008312  0.008667  0.016979
more_than_80   0.069490  0.114083  0.183573
All            0.419500  0.580500  1.000000
 
# of rows: 
 1    54231
0     6791
Name: Calcium_Avg_Ever_cat, dtype: int64
# of outliers:

### Run LOGISTICS REGRESSION

In [280]:
# extract needed columns
osteo_extract_filter = data_osteo.filter(['Strata','sex','Race_Cat_combine', 'Age_combine','osteo_predict', 'BMI_Avg_imputed', 
     'Alcohol_Prior', 'Tobacco_Prior','Drug_antipsych_prior', 'Drug_Estrogens_prior', 'Drug_Glucocorticoids_prior',
    'Drug_Nsaids_prior', 'Drug_Opiates_prior', 'Drug_Thiazide_prior','Drug_Loop_Diuretic_Prior',
      'Drug_Pp_inhibitors_prior',  'Drug_Progesterone_prior',   'Drug_Seizure_prior',  'Drug_Ssris_prior',
 'Drug_Tc_antidepress_prior',  'HeartDisease_Prior',  'Liver_Prior', 'PulmDisease_Prior',  'CNS_Disease_Prior',                                               
 'Malignancy_Prior','Hyponatremia_Prior','Chronic_Hyponatremia', 'Recent_Hyponatremia',
                                          
    'Calcium_Closest_Osteo_decile','Calcium_Avg_Prior_decile','Calcium_Avg_Ever_decile','Sodium_Closest_Osteo_decile',
    
    'Sodium_Avg_Prior_decile','Sodium_Avg_Ever_decile','Sodium_Worst_Prior_decile','Sodium_Worst_Ever_decile',
    'Calcium_Closest_Osteo_cat', 'Calcium_Avg_Prior_cat','Calcium_Avg_Ever_cat','Sodium_Avg_Prior_cat',
   'Sodium_Avg_Ever_cat','Sodium_Worst_Prior_cat','Sodium_Worst_Ever_cat']) 

osteo_extract_filter.shape

(61022, 43)

In [281]:
osteo_extract_filter.columns

Index(['Strata', 'sex', 'Race_Cat_combine', 'Age_combine', 'osteo_predict',
       'BMI_Avg_imputed', 'Alcohol_Prior', 'Tobacco_Prior',
       'Drug_antipsych_prior', 'Drug_Estrogens_prior',
       'Drug_Glucocorticoids_prior', 'Drug_Nsaids_prior', 'Drug_Opiates_prior',
       'Drug_Thiazide_prior', 'Drug_Loop_Diuretic_Prior',
       'Drug_Pp_inhibitors_prior', 'Drug_Progesterone_prior',
       'Drug_Seizure_prior', 'Drug_Ssris_prior', 'Drug_Tc_antidepress_prior',
       'HeartDisease_Prior', 'Liver_Prior', 'PulmDisease_Prior',
       'CNS_Disease_Prior', 'Malignancy_Prior', 'Hyponatremia_Prior',
       'Chronic_Hyponatremia', 'Recent_Hyponatremia',
       'Calcium_Closest_Osteo_decile', 'Calcium_Avg_Prior_decile',
       'Calcium_Avg_Ever_decile', 'Sodium_Closest_Osteo_decile',
       'Sodium_Avg_Prior_decile', 'Sodium_Avg_Ever_decile',
       'Sodium_Worst_Prior_decile', 'Sodium_Worst_Ever_decile',
       'Calcium_Closest_Osteo_cat', 'Calcium_Avg_Prior_cat',
       'Calcium_Avg_Ever_

In [189]:
osteo_extract_filter.dtypes

Strata                            int64
sex                              object
Race_Cat_combine                 object
Age_combine                      object
osteo_predict                     int64
BMI_Avg_imputed                 float64
Alcohol_Prior                     int64
Tobacco_Prior                     int64
Drug_antipsych_prior              int64
Drug_Estrogens_prior              int64
Drug_Glucocorticoids_prior        int64
Drug_Nsaids_prior                 int64
Drug_Opiates_prior                int64
Drug_Thiazide_prior               int64
Drug_Loop_Diuretic_Prior          int64
Drug_Pp_inhibitors_prior          int64
Drug_Progesterone_prior           int64
Drug_Seizure_prior                int64
Drug_Ssris_prior                  int64
Drug_Tc_antidepress_prior         int64
HeartDisease_Prior                int64
Liver_Prior                       int64
PulmDisease_Prior                 int64
CNS_Disease_Prior                 int64
Malignancy_Prior                  int64


## OHE

In [ ]:
#need to convert to object datatype
    'Calcium_Closest_Osteo_decile', 'Calcium_Avg_Prior_decile','Calcium_Avg_Ever_decile',
    'Sodium_Closest_Osteo_decile', 'Sodium_Avg_Prior_decile','Sodium_Avg_Ever_decile','Sodium_Worst_Prior_decile',
    'Sodium_Worst_Ever_decile'
    'Calcium_Closest_Osteo_cat', 'Calcium_Avg_Prior_cat','Calcium_Avg_Ever_cat',
   'Sodium_Avg_Prior_cat', 
   'Sodium_Avg_Ever_cat',
    'Sodium_Worst_Prior_cat',
    'Sodium_Worst_Ever_cat'

In [282]:
def convert_to_str(data, var):  
    print('before convert: ', data[var].dtypes)
    data[var] = data[var].astype(str)
    print('after convert: ', data[var].dtypes)

In [283]:
var = ['Calcium_Closest_Osteo_decile', 'Calcium_Avg_Prior_decile','Calcium_Avg_Ever_decile',
    'Sodium_Avg_Prior_decile','Sodium_Avg_Ever_decile','Sodium_Worst_Prior_decile','Sodium_Worst_Ever_decile',  
       'Sodium_Closest_Osteo_decile',
    'Calcium_Closest_Osteo_cat', 'Calcium_Avg_Prior_cat','Calcium_Avg_Ever_cat',
   'Sodium_Avg_Prior_cat','Sodium_Avg_Ever_cat','Sodium_Worst_Prior_cat','Sodium_Worst_Ever_cat']
for i in var: 
    print(i)
    convert_to_str(osteo_extract_filter,i)
    print(' ')

Calcium_Closest_Osteo_decile
before convert:  float64
after convert:  object
 
Calcium_Avg_Prior_decile
before convert:  float64
after convert:  object
 
Calcium_Avg_Ever_decile
before convert:  float64
after convert:  object
 
Sodium_Avg_Prior_decile
before convert:  float64
after convert:  object
 
Sodium_Avg_Ever_decile
before convert:  float64
after convert:  object
 
Sodium_Worst_Prior_decile
before convert:  float64
after convert:  object
 
Sodium_Worst_Ever_decile
before convert:  float64
after convert:  object
 
Sodium_Closest_Osteo_decile
before convert:  int64
after convert:  object
 
Calcium_Closest_Osteo_cat
before convert:  int64
after convert:  object
 
Calcium_Avg_Prior_cat
before convert:  int64
after convert:  object
 
Calcium_Avg_Ever_cat
before convert:  int64
after convert:  object
 
Sodium_Avg_Prior_cat
before convert:  int64
after convert:  object
 
Sodium_Avg_Ever_cat
before convert:  int64
after convert:  object
 
Sodium_Worst_Prior_cat
before convert:  int64
af

In [285]:
# Copy the original data
data_OHE = osteo_extract_filter.copy()

numeric_subset = data_OHE.select_dtypes('number')
categorical_subset = data_OHE.select_dtypes('object')

# One hot encode
categorical_subset = pd.get_dummies(categorical_subset) 

# Join the two dataframes using concat. Make sure to use axis = 1 to perform a column bind
data_OHE = pd.concat([numeric_subset, categorical_subset], axis = 1)

print(data_OHE.shape)

(61022, 137)


In [286]:
data_OHE.columns

Index(['Strata', 'osteo_predict', 'BMI_Avg_imputed', 'Alcohol_Prior',
       'Tobacco_Prior', 'Drug_antipsych_prior', 'Drug_Estrogens_prior',
       'Drug_Glucocorticoids_prior', 'Drug_Nsaids_prior', 'Drug_Opiates_prior',
       ...
       'Calcium_Avg_Ever_cat_0', 'Calcium_Avg_Ever_cat_1',
       'Sodium_Avg_Prior_cat_0', 'Sodium_Avg_Prior_cat_1',
       'Sodium_Avg_Ever_cat_0', 'Sodium_Avg_Ever_cat_1',
       'Sodium_Worst_Prior_cat_0', 'Sodium_Worst_Prior_cat_1',
       'Sodium_Worst_Ever_cat_0', 'Sodium_Worst_Ever_cat_1'],
      dtype='object', length=137)

## LR

In [287]:
fold = KFold(10, shuffle = True, random_state = 12345)

strata = data_OHE['Strata'].unique() 

all_preds = np.full(data_OHE.shape[0], 100)
probability = np.ones(data_OHE.shape[0])

for i, (train_index, test_index) in enumerate(fold.split(strata)):
    
    train_index_strata = strata[train_index]
    test_index_strata = strata[test_index] 
    
    X_train = data_OHE.loc[data_OHE['Strata'].isin(train_index_strata)].drop('osteo_predict', axis = 1)
    X_test = data_OHE.loc[data_OHE['Strata'].isin(test_index_strata)].drop('osteo_predict', axis = 1)
     
    y_train = data_OHE.loc[data_OHE['Strata'].isin(train_index_strata)]['osteo_predict']
    y_test = data_OHE.loc[data_OHE['Strata'].isin(test_index_strata)]['osteo_predict']

    lr = linear_model.LogisticRegression() 
    fit = lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_test)
    all_preds[y_test.index] = y_pred
    
    probs = lr.predict_proba(X_test)
    probability[y_test.index] = probs[:, 1]
 
#     print(classification_report(y_test, y_pred))

In [288]:
print('final result: ')
print(classification_report(data_OHE['osteo_predict'], all_preds), '\n')
print(confusion_matrix(data_OHE['osteo_predict'], all_preds)) 
print('ROC_AUC: ', roc_auc_score(data_OHE['osteo_predict'], probability))

final result: 
             precision    recall  f1-score   support

          0       0.71      0.79      0.75     30511
          1       0.76      0.68      0.72     30511

avg / total       0.73      0.73      0.73     61022
 

[[23980  6531]
 [ 9873 20638]]
ROC_AUC:  0.8189676459172313


## RANDOM FOREST

In [290]:
fold = KFold(10, shuffle = True, random_state = 12345)

strata = data_OHE['Strata'].unique() 

all_preds = np.full(data_OHE.shape[0], 100)
probability = np.ones(data_OHE.shape[0])

for i, (train_index, test_index) in enumerate(fold.split(strata)):
    
    train_index_strata = strata[train_index]
    test_index_strata = strata[test_index] 
    
    X_train = data_OHE.loc[data_OHE['Strata'].isin(train_index_strata)].drop('osteo_predict', axis = 1)
    X_test = data_OHE.loc[data_OHE['Strata'].isin(test_index_strata)].drop('osteo_predict', axis = 1)
     
    y_train = data_OHE.loc[data_OHE['Strata'].isin(train_index_strata)]['osteo_predict']
    y_test = data_OHE.loc[data_OHE['Strata'].isin(test_index_strata)]['osteo_predict']

    clf=RandomForestClassifier(n_estimators=100, random_state = 12345)
    fit = clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    all_preds[y_test.index] = y_pred
    
    probs = clf.predict_proba(X_test)
    probability[y_test.index] = probs[:, 1]
 
#     print(classification_report(y_test, y_pred))

In [291]:
print('final result: ')
print(classification_report(data_OHE['osteo_predict'], all_preds), '\n')
print(confusion_matrix(data_OHE['osteo_predict'], all_preds)) 
print('ROC_AUC: ', roc_auc_score(data_OHE['osteo_predict'], probability))

final result: 
             precision    recall  f1-score   support

          0       0.73      0.79      0.76     30511
          1       0.77      0.70      0.74     30511

avg / total       0.75      0.75      0.75     61022
 

[[24225  6286]
 [ 9025 21486]]
ROC_AUC:  0.8369783501775335


In [292]:
import pandas as pd

# Extract feature importances
fi = pd.DataFrame({'feature': list(data_OHE.drop('osteo_predict', axis = 1).columns),
                   'importance': clf.feature_importances_}).\
                    sort_values('importance', ascending = False)

# Display
fi.iloc[:20, :]

,feature,importance
1,BMI_Avg_imputed,0.106613
0,Strata,0.092746
56,Calcium_Avg_Ever_decile_0.0,0.038281
127,Calcium_Avg_Ever_cat_1,0.022041
126,Calcium_Avg_Ever_cat_0,0.019846
66,Calcium_Avg_Ever_decile_nan,0.019364
125,Calcium_Avg_Prior_cat_1,0.016181
124,Calcium_Avg_Prior_cat_0,0.013077
22,Chronic_Hyponatremia,0.012940
30,Age_combine_from_50_to_70,0.012450
